SUPERVISED REGRESSION


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge, Lasso
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import zipfile
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

In [2]:
# Load data
with zipfile.ZipFile('production-quality.zip', 'r') as zip_ref:
    zip_ref.extractall('production-quality')

In [3]:
# Read data
X_init = pd.read_csv("production-quality/data_X.csv")
submission = pd.read_csv("production-quality/sample_submission.csv")
Y_init = pd.read_csv("production-quality/data_Y.csv")

In [4]:
# Convert to date_time format
X_init['date_time']=pd.to_datetime(X_init['date_time'])
Y_init['date_time']=pd.to_datetime(Y_init['date_time'])
submission['date_time']=pd.to_datetime(submission['date_time'])

In [5]:
# extracting only the date and hour
X_init['date_hour']=X_init['date_time'].dt.strftime('%Y-%m-%d %H')
X_init=X_init.drop(columns=['date_time'])

In [6]:
# Grouping the data by date_hour and AH_data and aggregating it into lists
X_aggregated = X_init.groupby(['date_hour', 'AH_data']).agg(lambda x: list(x)).reset_index()
X_aggregated.head()

,date_hour,AH_data,T_data_1_1,T_data_1_2,T_data_1_3,T_data_2_1,T_data_2_2,T_data_2_3,T_data_3_1,T_data_3_2,T_data_3_3,T_data_4_1,T_data_4_2,T_data_4_3,T_data_5_1,T_data_5_2,T_data_5_3,H_data
0,2015-01-01 00,9.22,"[212, 212, 212, 213, 213, 213, 213, 213, 213, ...","[210, 211, 211, 211, 211, 211, 212, 212, 212, ...","[211, 211, 211, 211, 211, 211, 211, 211, 211, ...","[347, 346, 345, 344, 343, 342, 341, 340, 339, ...","[353, 352, 352, 351, 350, 350, 349, 348, 347, ...","[347, 346, 346, 346, 346, 346, 346, 345, 345, ...","[474, 475, 476, 477, 478, 479, 480, 482, 483, ...","[473, 473, 473, 473, 473, 473, 473, 473, 473, ...","[481, 481, 481, 481, 482, 482, 482, 482, 482, ...","[346, 349, 352, 355, 358, 360, 363, 365, 367, ...","[348, 348, 349, 349, 349, 349, 350, 350, 350, ...","[355, 355, 355, 355, 355, 354, 354, 354, 354, ...","[241, 241, 242, 242, 243, 243, 244, 244, 244, ...","[241, 241, 241, 241, 241, 241, 241, 241, 241, ...","[243, 243, 242, 242, 242, 242, 242, 242, 242, ...","[167.85, 162.51, 164.99, 167.34, 163.04, 163.8..."
1,2015-01-01 01,7.82,"[210, 210, 210, 210, 209, 209, 209, 209, 209, ...","[213, 213, 212, 212, 212, 212, 212, 212, 211, ...","[212, 212, 212, 212, 212, 212, 212, 212, 212, ...","[299, 299, 298, 298, 297, 297, 296, 296, 296, ...","[300, 299, 298, 297, 296, 295, 293, 292, 291, ...","[346, 346, 346, 347, 347, 347, 347, 347, 348, ...","[523, 523, 524, 524, 524, 525, 525, 526, 526, ...","[473, 473, 473, 473, 473, 473, 473, 473, 474, ...","[490, 490, 490, 490, 490, 491, 491, 491, 491, ...","[330, 329, 328, 327, 326, 326, 325, 325, 325, ...","[356, 357, 357, 357, 357, 357, 357, 357, 357, ...","[346, 346, 345, 345, 345, 345, 345, 345, 345, ...","[244, 243, 243, 243, 242, 242, 241, 241, 240, ...","[240, 240, 240, 240, 240, 240, 240, 240, 240, ...","[238, 238, 238, 238, 238, 238, 238, 238, 238, ...","[160.3, 164.12, 168.02, 166.28, 162.73, 165.65..."
2,2015-01-01 02,6.03,"[213, 214, 214, 214, 215, 215, 215, 216, 216, ...","[193, 192, 192, 191, 191, 190, 190, 189, 189, ...","[212, 212, 211, 211, 211, 211, 211, 211, 211, ...","[285, 285, 285, 285, 285, 285, 285, 285, 285, ...","[262, 263, 264, 265, 266, 268, 269, 270, 272, ...","[363, 362, 362, 362, 361, 361, 361, 360, 359, ...","[527, 527, 526, 526, 526, 525, 525, 524, 524, ...","[481, 481, 482, 482, 482, 483, 483, 484, 484, ...","[490, 490, 490, 490, 490, 490, 490, 490, 490, ...","[366, 366, 366, 366, 366, 366, 366, 366, 366, ...","[351, 350, 350, 350, 350, 349, 349, 349, 349, ...","[342, 342, 342, 342, 342, 342, 342, 343, 343, ...","[239, 239, 239, 239, 239, 239, 239, 239, 239, ...","[237, 237, 236, 236, 236, 235, 235, 234, 234, ...","[239, 239, 239, 239, 239, 239, 239, 239, 239, ...","[166.75, 167.87, 166.66, 164.4, 165.5, 161.84,..."
3,2015-01-01 03,8.36,"[248, 248, 247, 246, 245, 244, 243, 242, 241, ...","[207, 208, 210, 211, 212, 213, 215, 216, 217, ...","[212, 212, 212, 212, 212, 213, 213, 213, 214, ...","[303, 304, 304, 305, 306, 306, 307, 308, 308, ...","[271, 263, 256, 248, 241, 234, 227, 221, 214, ...","[331, 331, 332, 333, 333, 334, 335, 335, 336, ...","[486, 485, 484, 483, 482, 480, 479, 478, 477, ...","[488, 488, 488, 487, 486, 486, 485, 485, 484, ...","[482, 482, 482, 482, 482, 482, 482, 482, 482, ...","[347, 347, 346, 346, 345, 345, 344, 344, 344, ...","[344, 345, 345, 345, 345, 345, 346, 346, 346, ...","[346, 346, 346, 346, 346, 346, 346, 347, 347, ...","[235, 235, 235, 235, 235, 235, 235, 235, 235, ...","[235, 235, 236, 236, 236, 236, 237, 237, 237, ...","[238, 237, 237, 237, 237, 237, 237, 237, 237, ...","[163.84, 164.69, 166.28, 165.3, 166.4, 166.26,..."
4,2015-01-01 04,7.02,"[232, 232, 232, 233, 233, 233, 234, 234, 234, ...","[229, 229, 230, 230, 231, 231, 232, 232, 233, ...","[239, 239, 240, 241, 241, 242, 242, 243, 243, ...","[355, 356, 357, 358, 358, 359, 360, 360, 360, ...","[152, 156, 160, 165, 169, 174, 178, 182, 186, ...","[349, 349, 349, 349, 349, 348, 348, 348, 348, ...","[433, 433, 432, 432, 431, 431, 430, 430, 429, ...","[467, 468, 

In [7]:
# Expands the columns that has lists into seperate columns
for col in X_aggregated.columns:
    if X_aggregated[col].apply(lambda x: isinstance(x, list)).any():
        # Expanding only the rows with lists into separate columns
        expanded_cols = X_aggregated[col].apply(lambda x: x if isinstance(x, list) else [x]).apply(pd.Series)
        # Each new column is named with a suffix indicating position in the list
        expanded_cols.columns = [f"{col}_{i+1}" for i in range(expanded_cols.shape[1])]
        # Concatenate expanded columns with the original DataFrame
        X_aggregated = pd.concat([X_aggregated.drop(columns=[col]), expanded_cols], axis=1)
X_aggregated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35065 entries, 0 to 35064
Columns: 962 entries, date_hour to H_data_60
dtypes: float64(961), object(1)
memory usage: 257.4+ MB


In [8]:
# Subtract 1 hour ( Y has produced the quality result of the work done in previous hour, so we shift Y to match the values of that hour)
Y_init['date_time'] = Y_init['date_time'] - pd.Timedelta(hours=1)
# Converting into date hour format
Y_init['date_hour'] = Y_init['date_time'].dt.strftime('%Y-%m-%d %H')
Y_init = Y_init.drop(columns=['date_time'])
submission['date_hour'] = submission['date_time'].dt.strftime('%Y-%m-%d %H')
submission_file = submission.drop(columns=['date_time'])
Y_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29184 entries, 0 to 29183
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   quality    29184 non-null  int64 
 1   date_hour  29184 non-null  object
dtypes: int64(1), object(1)
memory usage: 456.1+ KB


In [9]:
# Merge X and Y - Inner 
data = pd.merge(X_aggregated, Y_init, on='date_hour', how='inner')
data['date_hour'] = pd.to_datetime(data['date_hour'], format='%Y-%m-%d %H')
data['year'] = data['date_hour'].dt.year
data['month'] = data['date_hour'].dt.month
data['day'] = data['date_hour'].dt.day
data['hour'] = data['date_hour'].dt.hour
data = data.drop(columns=['date_hour'])

# Merge X and submission file
validation = pd.merge(X_aggregated, submission[["date_hour","quality"]], left_on="date_hour", right_on="date_hour", how='inner')
validation['date_hour'] = pd.to_datetime(validation['date_hour'], format='%Y-%m-%d %H')
validation['year'] = validation['date_hour'].dt.year
validation['month'] = validation['date_hour'].dt.month
validation['day'] = validation['date_hour'].dt.day
validation['hour'] = validation['date_hour'].dt.hour
validation = validation.drop(columns=['date_hour'])

PCA - To reduce the high dimensional dataset

In [10]:
# There are 962 columns after grouping and expanding the columns, Thus, we use PCA to reduce the dimension of the features to improve computational efficiency
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X=data.drop(columns=['quality'])
y=data['quality']
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pca = PCA(n_components=0.95)
X_pca = pca.fit_transform(X_scaled)

MODEL TRAINING

In [11]:
X=data.drop(columns=['quality'])
y=data['quality']
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=0)
models = {
    'linear': LinearRegression(),
    'Ridge': Ridge(),
    'Lasso': Lasso(),
    'RandomForest': RandomForestRegressor(n_estimators=200, random_state=42),
    'GradientBoosting': GradientBoostingRegressor(n_estimators=200, random_state=42),
    'Support Vector Machine (SVM)': SVR(),
    'Decision Tree': DecisionTreeRegressor(random_state=42),
    'XGBoost': XGBRegressor(n_estimators=200, random_state=42)
}

In [12]:
model_scores={}
print("Model Performance Comparison (Mean Absolute Error):")
for model_name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    score = mean_absolute_error(y_test, y_pred)
    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    model_scores[model_name] = score
    print(f"{model_name} MAE : {score} MAPE : {mape}")
    
    
best_model_name = min(model_scores, key=model_scores.get)
best_model = models[best_model_name]
print(f"\nBest Model: {best_model_name}\n ")

Model Performance Comparison (Mean Absolute Error):
linear MAE : 13.113019374303367 MAPE : 3.430806933982304
Ridge MAE : 12.87013473832554 MAPE : 3.3720772609647813
Lasso MAE : 13.36495517424279 MAPE : 3.5212465649862286
RandomForest MAE : 7.6849777282850775 MAPE : 1.9981977315160566
GradientBoosting MAE : 7.819580574061928 MAPE : 2.0225741167136357
Support Vector Machine (SVM) MAE : 11.460324005714757 MAPE : 3.023652696854505
Decision Tree MAE : 12.73239677916738 MAPE : 3.3019262033555554
XGBoost MAE : 7.83484349161749 MAPE : 2.0277039591492008

Best Model: RandomForest
 


In [13]:
# We found the best model as XGBoost with MAE = 7 after training, Thus we use it to generate results for our submission file.
val = validation.drop(columns=['quality'])
val_scaled = scaler.transform(val)
val_pca = pca.transform(val_scaled)
quality=best_model.predict(val_scaled)
submission['quality']=quality

In [14]:
submission.to_csv('production-quality/sample_submission.csv', index=False)

In [15]:
submission = submission[["date_time","quality"]]
submission

,date_time,quality
0,2018-05-04 00:05:00,440.875
1,2018-05-04 01:05:00,435.670
2,2018-05-04 02:05:00,407.890
3,2018-05-04 03:05:00,408.670
4,2018-05-04 04:05:00,413.140
...,...,...
5803,2018-12-31 19:05:00,472.105
5804,2018-12-31 20:05:00,453.770
5805,2018-12-31 21:05:00,443.870
5806,2018-12-31 22:05:00,439.100
